In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

In [2]:
#数据准备
raw_data = pd.read_csv("D:/Code/Jupyter-notebook/data/MVP/NBA Players - Basic Season Stats (1950-2017).csv")

raw_data = raw_data.drop(['#',' Player Salary in $ ','PER', 'TS%', '3PAr', 'FTr', 'ORB%',
       'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'blanl', 'OWS',
       'DWS', 'WS', 'WS/48', 'blank2', 'OBPM', 'DBPM', 'BPM', 'VORP'],axis=1)
raw_data = raw_data.fillna(0)
#筛选出打了40场以上且场均25min以上且得分超过10分的球员
raw_data = raw_data[raw_data['MP']>25*50]    #常规赛82场，但考虑缩水赛季为50场
raw_data = raw_data[raw_data['G']>40]
raw_data = raw_data[raw_data['PTS']>15*50]
#############################
#修正：增加MVP个数 
#for row in range(raw_data.shape[0]):
#    if raw_data.iloc[row,-2] > 2050:   #MVP平均分25*82场
#        raw_data.iloc[row,-1] = 1
#raw_data[raw_data['MVP'] == 1].shape

D:\Anaconda\anaconda\envs\pytorch\lib\site-packages\IPython\core\interactiveshell.py:3044: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#按年份区分数据 
year_data = {}
cur_year = 2017
cur_count_rows = 0
pre_count_rows = 0
for year in range(raw_data.shape[0]):
    if raw_data.iloc[year,0] < 1955:    #去掉50-55年数据 
        pass
    elif raw_data.iloc[year,0] - cur_year == 0:
        cur_count_rows += 1
    else:
        if (raw_data.iloc[year,0]+1) not in year_data:
            year_data[raw_data.iloc[year,0]+1] = raw_data.iloc[pre_count_rows:cur_count_rows,:]
        pre_count_rows = cur_count_rows
        cur_year -= 1
        cur_count_rows += 1



In [4]:
#划分数据集，划分出10年做验证
train_data = year_data[2007].iloc[:,5:-1]
train_label = year_data[2007].iloc[:,-1]

test_data = year_data[2017].iloc[:,5:-1]
test_label = year_data[2017].iloc[:,-1]


for year in year_data.keys():
    if year not in [2007,2017]:
        if year < 2008:
            train_data = pd.concat([train_data,year_data[year].iloc[:,5:-1]],axis=0)
            train_label = pd.concat([train_label,year_data[year].iloc[:,-1]],axis=0)
        else:
            test_data = pd.concat([test_data,year_data[year].iloc[:,5:-1]],axis=0)
            test_label = pd.concat([test_label,year_data[year].iloc[:,-1]],axis=0)



In [5]:
year_data[2017].columns


Index(['Season Start', 'Player Name', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP',
       'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%',
       'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV',
       'PF', 'PTS', 'MVP'],
      dtype='object')

In [6]:
#数据正则化
for j in range(train_data.shape[1]):
    Mean = np.mean(train_data.iloc[:,j])
    Std = np.std(train_data.iloc[:,j])
    for i in range(train_data.shape[0]):
        train_data.iloc[i,j] = (train_data.iloc[i,j] - Mean) / Std

for j in range(test_data.shape[1]):
    Mean = np.mean(test_data.iloc[:,j])
    Std = np.std(test_data.iloc[:,j])
    for i in range(test_data.shape[0]):
        test_data.iloc[i,j] = (test_data.iloc[i,j] - Mean) / Std

In [7]:
train_data.shape[1]

25

In [8]:
#建模
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.linear = nn.Linear(25,2)
        
    def forward(self,x):
        out = self.linear(x)
        out = torch.sigmoid(out)
        return out 

In [9]:
#计算模型准确率
def test(pred_lab,lab):
    t = pred_lab.max(-1)[1] == lab
    return torch.mean(t.float())
    

In [10]:
net = Net()
criterion = nn.CrossEntropyLoss()
optm = torch.optim.Adam(net.parameters(),lr=0.01)
epochs = 10000

In [11]:
#Training
for epoch in range(epochs):
    net.train()
    train_x = torch.from_numpy(train_data.values)
    train_y = torch.from_numpy(train_label.values)
    y_hat = net(train_x.float())

    loss = criterion(y_hat,train_y)
    optm.zero_grad()
    loss.backward()
    optm.step()
    
    if (epoch+1)%500 == 0:
        net.eval()
        test_x = torch.from_numpy(test_data.values)
        test_y = torch.from_numpy(test_label.values)
        test_out = net(test_x.float())
        
        accu = test(test_out,test_y)
        print("Epoch:{},Loss:{:.4f},Accuracy：{:.4f}".format(epoch+1,loss.item(),accu))
        
        

Epoch:500,Loss:0.3505,Accuracy：0.9923
Epoch:1000,Loss:0.3329,Accuracy：0.9923
Epoch:1500,Loss:0.3283,Accuracy：0.9923
Epoch:2000,Loss:0.3262,Accuracy：0.9923
Epoch:2500,Loss:0.3250,Accuracy：0.9923
Epoch:3000,Loss:0.3241,Accuracy：0.9923
Epoch:3500,Loss:0.3236,Accuracy：0.9923
Epoch:4000,Loss:0.3231,Accuracy：0.9923
Epoch:4500,Loss:0.3228,Accuracy：0.9923
Epoch:5000,Loss:0.3225,Accuracy：0.9931
Epoch:5500,Loss:0.3222,Accuracy：0.9931
Epoch:6000,Loss:0.3220,Accuracy：0.9931
Epoch:6500,Loss:0.3218,Accuracy：0.9931
Epoch:7000,Loss:0.3217,Accuracy：0.9931
Epoch:7500,Loss:0.3216,Accuracy：0.9931
Epoch:8000,Loss:0.3214,Accuracy：0.9931
Epoch:8500,Loss:0.3213,Accuracy：0.9931
Epoch:9000,Loss:0.3212,Accuracy：0.9931
Epoch:9500,Loss:0.3211,Accuracy：0.9931
Epoch:10000,Loss:0.3210,Accuracy：0.9931


In [19]:
#预测2018年MVP
pred_data = pd.read_csv('D:/Code/Jupyter-notebook/data/MVP/NBA Players - Basic Season Stats(2018).csv')
#pred_data = pred_data[pred_data['PTS']>15*50]
#pred_data = pred_data[pred_data['MP']>25*50]
#pred_data = pred_data[pred_data['G']>40]
pred_data


for j in range(pred_data.shape[1]-5):
    Mean = np.mean(pred_data.iloc[:,j+5])
    Std = np.std(pred_data.iloc[:,j+5])
    for i in range(pred_data.shape[0]):
        pred_data.iloc[i,j+5] = (pred_data.iloc[i,j+5] - Mean) / Std

pred_data

#因为MVP数据太多，导致预测不准

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Alex Abrines,SG,24,OKC,1.130726,-0.450887,0.174929,-0.286165,-0.178229,...,0.706184,-0.258758,-0.346717,-0.334586,-0.570333,0.201705,-0.435199,-0.528337,0.641068,-0.184700
1,2,Quincy Acy,PF,27,BRK,0.950600,-0.450887,0.443339,-0.195471,0.032245,...,0.496582,0.015930,0.561623,0.431831,-0.325199,0.044154,0.379158,0.064605,1.014230,-0.055990
2,3,Steven Adams,C,24,OKC,1.166751,2.094498,1.788966,1.727242,1.019197,...,-1.247845,6.765407,1.153101,2.725723,-0.063158,1.903258,2.279322,1.216606,1.999380,1.375354
3,4,Bam Adebayo,C,20,MIA,0.914575,-0.039134,0.454075,0.070565,-0.038861,...,-0.152507,1.546335,0.885527,1.096417,0.046730,0.012644,0.844504,0.166252,0.850039,0.090473
4,5,Arron Afflalo,SG,32,ORL,0.338173,-0.638048,-0.364276,-0.588478,-0.545136,...,0.692661,-0.690410,-0.529793,-0.591845,-0.553427,-0.869643,-0.396420,-0.596102,-0.373935,-0.570830
5,6,Cole Aldrich,C,29,MIN,-0.814631,-0.750344,-1.119402,-0.951254,-0.963240,...,-2.775909,-0.710031,-0.881863,-0.865183,-0.781656,-0.932664,-0.706651,-0.934926,-1.045627,-0.941425
6,7,LaMarcus Aldridge,C,32,SAS,1.130726,2.057065,1.815210,3.172300,2.825291,...,0.631809,4.057768,1.772744,2.457745,0.477828,0.359256,2.744669,0.928606,1.193348,2.882148
7,8,Jarrett Allen,C,19,BRK,1.022651,0.410052,0.541159,0.433341,0.123261,...,0.219367,2.056470,0.751741,1.133934,-0.392822,-0.113397,2.667111,0.437311,0.984377,0.334578
8,9,Kadeem Allen,PG,25,BOS,-0.922706,-0.712912,-1.050212,-0.945208,-0.943330,...,0.232889,-0.690410,-0.917070,-0.886621,-0.705580,-0.901153,-0.667872,-0.799396,-0.985921,-0.925891
9,10,Tony Allen,SF,36,NOP,-0.778606,-0.750344,-0.852185,-0.715450,-0.747077,...,-1.484492,-0.376481,-0.776242,-0.693677,-0.730939,-0.649071,-0.629093,-0.629984,-0.478420,-0.739484


In [20]:
x = torch.from_numpy(pred_data.iloc[:,5:].values)
result = net(x.float())

##返回位置
pos = []
for i in range(len(result)):
    if result.max(-1)[1][i] == 1:
        pos.append(i)
result
pos

[148, 171, 232, 269, 303, 591, 632]

In [21]:
#把原始数据中每年MVP候选人全定为1
for j in pos:
    MVPname = pred_data.iloc[j,1]
    print(MVPname)

Anthony Davis
Kevin Durant
James Harden
Jrue Holiday
LeBron James
Karl-Anthony Towns
Jacob Wiley


In [22]:
#预测2019年的MVP
pred_data = pd.read_csv('D:/Code/Jupyter-notebook/data/MVP/NBA Players - Basic Season Stats(2019).csv')


for j in range(pred_data.shape[1]-5):
    Mean = np.mean(pred_data.iloc[:,j+5])
    Std = np.std(pred_data.iloc[:,j+5])
    for i in range(pred_data.shape[0]):
        pred_data.iloc[i,j+5] = (pred_data.iloc[i,j+5] - Mean) / Std
        
x = torch.from_numpy(pred_data.iloc[:,5:].values)
result = net(x.float())

##返回位置
pos = []
for i in range(len(result)):
    if result.max(-1)[1][i] == 1:
        pos.append(i)

for j in pos:
    MVPname = pred_data.iloc[j,1]
    print(MVPname)

Ryan Anderson
Giannis Antetokounmpo
Stephen Curry
Kevin Durant
Pau Gasol
James Harden
Alize Johnson
Zhaire Smith
Nikola Vucevic
